#Level A

>Creating a data descriptor that sets and returns values normally and prints a message logging their access.
>>The value of data descriptor is shared between all instances of a class

In [6]:
class Descriptor:

    def __init__(self):
        self.name = ''

    def __get__(self, instance, owner):
        print(f'Getting: {self.name}')
        return self.name

    def __set__(self, instance, name):
        print(f'Setting: {name}')
        self.name = name

    def __delete__(self, instance):
        print(f'Deleting: {self.name}')
        del self.name
        
class Person:
    name = Descriptor()

In [8]:
obj_1 = Person()
obj_2 = Person()

obj_1.name = 'Jack'
obj_2.name = 'John'

print(obj_1.name, obj_2.name)

Setting: Jack
Setting: John
Getting: John
Getting: John
John John


>Creating a data descriptor that sets and returns values normally and prints a message logging their access.
>>The value of data descriptor is unique for each instances of a class

In [9]:
class Descriptor:

    def __init__(self, field):
        self.field_name = field

    def __get__(self, instance, owner):
        print(f'Getting: {instance.__dict__[self.field_name]}')
        return instance.__dict__[self.field_name]

    def __set__(self, instance, value):
        print(f'Setting: {value}')
        instance.__dict__[self.field_name] = value

    def __delete__(self, instance):
        print(f'Deleting: {self.field_name}')
        del self.field_name


class Person:
    name = Descriptor('name')

    def __init__(self, name):
        self.name = name

In [10]:
obj_1 = Person('Jack')
obj_2 = Person('John')

print(obj_1.name, obj_2.name)

Setting: Jack
Setting: John
Getting: Jack
Getting: John
Jack John


#Level B

>Create a class Order with such data-attributes as shipping_price and order_price. The shipping and order price must be stored including VAT. Also, the tax calculated for Europe region and US region are different.

In [1]:
class Tax:
    taxes = {
        'EU': 0.2,
        'US':0.1
    }

    def __init__(self, country_post, field):
        self.tax = self.taxes[country_post]
        self.field_name = field

    def __get__(self, instance, owner):
        return instance.__dict__[self.field_name]

    def __set__(self, instance, value):
        instance.__dict__[self.field_name] = value * (1 + self.tax)


class EbayGermany:
    shipping_price = Tax('EU', 'shipping_price')
    order_price = Tax('EU', 'order_price')

    def __init__(self, **kwargs):
        self.shipping_price = kwargs['shipping_price']
        self.order_price = kwargs['order_price']


ebay_germany = EbayGermany(shipping_price=20, order_price=40)

print(ebay_germany.shipping_price)
print(ebay_germany.order_price)

24.0
48.0


>Interception undefined attribute in class and delegation them to the embedded object with the getattr built-in.

In [2]:
class Person:
    def __init__(self, full_name, job=None, pay=0):
        self.__name = full_name
        self.__job = job
        self.__pay = pay

    def last_name(self):
        return self.__name.split()[-1]

    def set_raise(self, percent):
        self.__pay = self.__pay * (1 + percent)

    def __str__(self):
        return f'[Person: {self.__name}, {self.__pay}]'


class Manager:
    def __init__(self, full_name, pay):
        self.__person = Person(full_name, 'mgr', pay)   # Embed a Person object

    def set_raise(self, percent, bonus=.10):
        self.__person.set_raise(percent + bonus)          # Intercept and delegate

    def __getattr__(self, attr):
        return getattr(self.__person, attr)             # Delegate all other attrs

    def __str__(self):
        return str(self.__person)


bob = Person('Bob Smith')
sue = Person('Sue Jones', job='dev', pay=100000)
print(bob)
print(sue)
print(bob.last_name(), sue.last_name())
sue.set_raise(.10)
print(sue)
tom = Manager('Tom Jones', 50000)  # Job name not needed:
tom.set_raise(.10)  # Implied/set by class
print(tom.last_name())
print(tom)

[Person: Bob Smith, 0]
[Person: Sue Jones, 100000]
Smith Jones
[Person: Sue Jones, 110000.00000000001]
Jones
[Person: Tom Jones, 60000.0]
